In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

C:\Users\egeca\AppData\Local\Temp\ipykernel_18244\3634875537.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
days = int(input("kaç günlük veri istiyorsunuz???"))
driver = webdriver.Chrome()
driver.get("https://www.mackolik.com/futbol/canli-sonuclar")
data = []
for day in range(days):
    dayButtonPath = f'/html/body/div[4]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/ol/li[{day + 4}]'
    button = driver.find_element(By.XPATH, dayButtonPath)
    button.click()
    time.sleep(1)
    main_div = driver.find_element(By.XPATH, '/html/body/div[4]/div/main/div[1]/div[1]/div[2]')
    wrappers = main_div.find_elements(By.XPATH, './div[not(div[contains(@class, "widget-gpt-ads")])]')
    for wrapper in wrappers:
        matches = wrapper.find_elements(By.XPATH, './div[not(contains(@class, "widget-livescore__title")) and not(contains(@class, "teads-inread"))]')
        for match in matches:
            try:
                match_url = match.find_element(By.CSS_SELECTOR, ".match-row__match-content").get_attribute("data-match-url")
                matchdata = []
                for element in match.find_elements(By.CSS_SELECTOR, ".match-row__match-content"):
                    matchdata.append(element.text)
                data_match_id = match.get_attribute('data-match-id')
                matchdata.append(data_match_id)
                matchdata.append(match_url)
                data.append(matchdata)
            except NoSuchElementException:
                continue
driver.quit()

kaç günlük veri istiyorsunuz??? 1


In [4]:
# '13:30\nMS\nAntalyaspor 2-2\nİstanbulspor\nİY 2-0'
# "23:00\n61'\nHeerenveen 0-1\nGo Ahead\nİY 0-0"
# '16:30', "26'", 'Ajax 1-0', 'NEC Nijmegen'
# '02:00\nA. San Luis -\nTijuana'
# '03:00\nERT\nGendrim -\nKsar'
df = []
for item in data:
    link = item[2],
    id = item[1]
    item = item[0]
    parcalar = item.split('\n')
    saat = parcalar[0]
    if (parcalar[1][-1]) == "-":
        # '02:00\nA. San Luis -\nTijuana'
        ic_saha = parcalar[1][:-1].strip()
        skor = parcalar[1][-1]
        deplasman = parcalar[2]
        durum = "-"
        if len(parcalar) == 4:
            # '03:00\nERT\nGendrim -\nKsar'
            durum = parcalar[1]
            ic_saha = parcalar[2][:-1].strip()
            deplasman = parcalar[3]
            skor = parcalar[2][-1]
        ilk_yari_skoru = "-"
    else:
        if len(parcalar) == 4:
            # '16:30', "26'", 'Ajax 1-0', 'NEC Nijmegen'
            durum = parcalar[1]
            ic_saha = parcalar[2][:-3].strip()
            skor = parcalar[2][-3:]
            deplasman = parcalar[3]
            ilk_yari_skoru = "-"
        else:
             # '13:30\nMS\nAntalyaspor 2-2\nİstanbulspor\nİY 2-0'
            durum = parcalar[1]
            ic_saha = parcalar[2][:-3].strip()
            skor = parcalar[2][-3:]
            deplasman = parcalar[3]
            ilk_yari_skoru = parcalar[4][-3:]
    df.append({
        'Saat': saat,
        'Durum': durum,
        'Skor': skor,
        'İç Saha': ic_saha,
        'Deplasman': deplasman,
        'İY': ilk_yari_skoru,
        'Link': link,
        'ID': id
    })
df = pd.DataFrame(df)

In [5]:
df

,Saat,Durum,Skor,İç Saha,Deplasman,İY,Link,ID
0,17:00,-,-,Adana Demir,Karagümrük,-,(https://www.mackolik.com/mac/adana-demirspor-...,tkvnqi6h0r0z0yjttc83t5w4
1,20:00,-,-,Antalyaspor,Gaziantep FK,-,(https://www.mackolik.com/mac/antalyaspor-vs-g...,tac66brtc2tjmngykioxvms4
2,20:00,-,-,Alanyaspor,Trabzonspor,-,(https://www.mackolik.com/mac/alanyaspor-vs-tr...,tvdfjstyz4s7iyi2ko9la978
3,14:00,-,-,Gençlerbirliği,Şanlıurfaspor,-,(https://www.mackolik.com/mac/gen%C3%A7lerbirl...,548z8w15cabvbq0k0lnl147x0
4,23:00,-,-,Sheffield Utd,Arsenal,-,(https://www.mackolik.com/mac/sheffield-united...,dz6if17j9xpjyqpezxbs85fkk
...,...,...,...,...,...,...,...,...
198,01:00,-,-,Sampaio Correa,Tuntum,-,(https://www.mackolik.com/mac/sampaio-correa-v...,9nkgjkf7h5caoisdw8s7matjo
199,04:00,-,-,Santa Ana,Guadalupe,-,(https://www.mackolik.com/mac/santa-ana-vs-gua...,79jybilgrz6ov2kgi63b8fgno
200,01:00,-,-,Cavalier,Humble,-,(https://www.mackolik.com/mac/cavalier-vs-humb...,1yl2g2gejhavswbqeiexpygpg
201,03:30,-,-,Dunbeholden,Harbour View,-,(https://www.mackolik.com/mac/dunbeholden-vs-h...,205ja2ooue9igp2eigcfqtl3o


In [42]:
oran1=[]
oran2=[]
oran3=[]
lists = [oran1,oran2,oran3]
for index, row in df.iterrows():
    if row['Durum'] == 'MS':
        continue
    driver = webdriver.Chrome()
    driver.get(row['Link'][0])
    time.sleep(0.5)
    try:
        cookiebutton = driver.find_element(By.XPATH, '/html/body/div[5]/a[2]')
        cookiebutton.click()
    except NoSuchElementException:
        continue
    button = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]/div/a[3]')
    button.click()
    piece = []
    categories = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/main/div[1]/div[2]/div/div[2]/div/div[1]/div/ul').find_elements(By.XPATH, './li[contains(@class, "widget-iddaa-markets__market-item")]')
    for category in range(len(categories)):
        for i in range(3):
            selected_list = lists[i]
            OranPATH = f'/html/body/div[4]/div[2]/main/div[1]/div[2]/div/div[2]/div/div[1]/div/ul/li[{category+1}]/div/div/ul[{i+2}]'
            selected_list.append(piece)
            piece = []
            for x in range(len(driver.find_element(By.XPATH, OranPATH).find_elements(By.XPATH, ".//li[not(a/img)]"))):
                OranPATH = f'/html/body/div[4]/div[2]/main/div[1]/div[2]/div/div[2]/div/div[1]/div/ul/li[{category+1}]/div/div/ul[{i+2}]/li[{x+2}]/a/span[2]'
                oran = driver.find_element(By.XPATH, OranPATH).get_attribute("textContent")
                piece.append(oran)     
    driver.close()

KeyboardInterrupt: 

In [43]:
print(oran1[1],oran2[0],oran3[0])

[' 2.12 ', ' 3.14 ', ' 2.52 '] [' 2.15 ', ' 3.19 ', ' 2.56 '] [' 2.11 ', ' 3.13 ', ' 2.51 ']


In [44]:
def listeDüz(list):
    for i in range(len(list)):
        try:
            list[i] = float(list[i])
        except ValueError:
            list[i] = 0
    return list

In [66]:
def max_ratio_sum(list1, list2, list3):
    max_sum = 0
    max_combination = []
    perm = permutations([0, 1, 2])
    for p in perm:
        sum_comb = float(list1[p[0]]) + float(list2[p[1]]) + float(list3[p[2]])
        if sum_comb >= max_sum:
            max_sum = sum_comb
            max_combination = [float(list1[p[0]]), float(list2[p[1]]), float(list3[p[2]])]
    return max_combination

In [76]:
def arbitrage(odds):
    # Toplam olasılığı hesapla
    total_probability = 1 / sum(1 / odd for odd in odds)

    # Eğer toplam olasılık 1'den büyükse arbitraj fırsatı vardır
    if total_probability > 1:
        # Potansiyel kazancı hesapla
        profit = total_probability - 1
        return True, profit
    else:
        return False

In [78]:
print(oran1[1],oran2[0],oran3[0])
print(max_ratio_sum(listeDüz(oran1[1]),listeDüz(oran2[0]),listeDüz(oran3[0])))

[2.12, 3.14, 2.52] [2.15, 3.19, 2.56] [2.11, 3.13, 2.51]
[2.52, 3.19, 2.11]


In [77]:
arbitrage = arbitrage(max_ratio_sum(listeDüz(oran1[1]),listeDüz(oran2[0]),listeDüz(oran3[0])))
if arbitrage:
    print(f"Arbitraj fırsatı bulundu! Potansiyel kazanç: {profit}")
else:
    print(arbitrage)

False


In [ ]:
Olasılıklar = [
    [MS1,MSX,MS2],
    [MS1,ÇX2],
    [MSX,Ç12],
    [MS2,Ç1X],
    [GOL<4.5,GOL>4.5],
    [GOL<1.5,GOL>1.5],
    [GOL<3.5,GOL>3.5],
    [GOL<0.5,GOL>0.5],
    [GOL<2.5,GOL>2.5],
    [1YARI<2.5,1YARI>2.5],
    [1YARI<0.5,1YARI>0.5],
    [YARI1,YARIÇX2],
    [YARIX,YARIÇ12],
    [YARI2,YARIÇ1X],
    [YARI1,YARIX,YARI2],
    [2YARI1,2YARIX,2YARI2],
    [TEK,ÇİFT],
    [KGVAR,KGYOK],
    [HNDMS2]

]